In [68]:
import torch                                                                    
import torch.nn as nn                                                           
import torch.nn.functional as F                                                 
import torch.optim as optim                                                     
import torchvision    

from numpy import array                                                         
from torchnlp.text_encoders import WhitespaceEncoder 
from torchnlp.utils import pad_tensor 

import matplotlib.pyplot as plt     

In [69]:
docs = ['China',                                                                
    'Italy',                                                                    
    'Germany',                                                                  
    'USA',                                                                      
    'Canada',                                                                   
    'Beijing',                                                                  
    'Rome',                                                                     
    'Berlin',                                                                   
    'Washington DC',                                                            
    'Ottawa']                                                                   
                  

In [70]:
labels = array([1,1,1,1,1,0,0,0,0,0])                                           
                                      

In [71]:
t  = WhitespaceEncoder( docs )     

In [72]:
t


In [73]:
t.vocab

['<pad>',
 '<unk>',
 '</s>',
 '<s>',
 '<copy>',
 'China',
 'Italy',
 'Germany',
 'USA',
 'Canada',
 'Beijing',
 'Rome',
 'Berlin',
 'Washington',
 'DC',
 'Ottawa']

In [74]:
encoded_docs =[ t.encode(x) for x in docs]   

In [75]:
encoded_docs

[tensor([5]),
 tensor([6]),
 tensor([7]),
 tensor([8]),
 tensor([9]),
 tensor([10]),
 tensor([11]),
 tensor([12]),
 tensor([13, 14]),
 tensor([15])]

In [76]:
max_length = 2                                                                  
padded_docs = [ pad_tensor(x, max_length) for x in encoded_docs ] 

In [77]:
padded_docs

[tensor([5, 0]),
 tensor([6, 0]),
 tensor([7, 0]),
 tensor([8, 0]),
 tensor([9, 0]),
 tensor([10,  0]),
 tensor([11,  0]),
 tensor([12,  0]),
 tensor([13, 14]),
 tensor([15,  0])]

In [78]:
t.vocab[0]

'<pad>'

In [187]:
y_train = torch.from_numpy(labels).float() 

In [188]:
y_train

tensor([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.])

In [193]:
y_train_1 = y_train.view(-1,1)

In [195]:
y_train_1

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [81]:
x_train =  torch.stack(padded_docs)

In [82]:
x_train

tensor([[ 5,  0],
        [ 6,  0],
        [ 7,  0],
        [ 8,  0],
        [ 9,  0],
        [10,  0],
        [11,  0],
        [12,  0],
        [13, 14],
        [15,  0]])

In [172]:
class MyNet(torch.nn.Module):                                                   
  # num_embeddings : vocab_size                                                 
  # embedding_dim:  the size of each embedding vector                           
  # n_out: final out, here we set 1 as we will use signoid for our 2-class      
  def __init__(self, num_embeddings, embedding_dim, max_length, n_out):                     
    super(MyNet, self).__init__()                                               
# embedding layers                                                              
# first parameter: size of the vocabulary                                       
# second parameter: output_dim, Dimension of the dense embedding.               
# We will choose a small embedding space of 2 dimensions for easy plotting      
    self.em = torch.nn.Embedding(num_embeddings, embedding_dim) 
    # n_out is 1 here for binary classification
    self.linear_1 = torch.nn.Linear( max_length * embedding_dim, n_out)                       
                                                                                
  def forward(self, x):                                                         
    y = self.em(x)      
    # need to flatten: squeeze
    y = y.view(y.size()[0], -1)
    y = self.linear_1(y)                       
    # sum on dim=1 should =1 as it is a probability of two classes, output softmax 
    y= F.sigmoid(y)                                                     
    return y                                                                    
                 

In [173]:
# Create a model                                                                
num_embeddings = t.vocab_size                                                     
embedding_dim = 2                                                               
n_out = 1  # two classes                                                                       
model = MyNet( num_embeddings, embedding_dim, max_length, n_out)                            
                                                                                
# Construct the loss function                                                   
# it will calculate softmax + CrossEntropy                                      
#criterion = torch.nn.CrossEntropyLoss()
# for binary classification
criterion = nn.BCELoss()                                                                      
# Construct the optimizer (Stochastic Gradient Descent in this case)            
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)                        
                                                                                
epochs = 5     

In [97]:
batch_size = 2 

In [129]:
x = x_train[0]

In [130]:
x

tensor([5, 0])

In [182]:
y= model.em(x)

In [183]:
y

tensor([[-0.8702, -0.4317],
        [-1.8287, -0.0838]], grad_fn=<EmbeddingBackward>)

In [177]:
y = y.view(y.size()[0], -1)

In [184]:
y.size()

torch.Size([2, 2])

In [170]:
y=y.squeeze()

In [171]:
y

tensor([-0.4078, -0.2813, -0.2527, -0.5317], grad_fn=<SqueezeBackward0>)

In [167]:
y = model.linear_1(y)  

RuntimeError: size mismatch, m1: [4 x 1], m2: [2 x 1] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:940

In [147]:
y

tensor([[-0.2087],
        [-0.2830]], grad_fn=<AddmmBackward>)

In [109]:
y1= F.sigmoid(y)    

In [60]:
y1

tensor([[-0.5212, -0.4911],
        [-0.9009, -0.9466]], grad_fn=<LogSoftmaxBackward>)

In [61]:
y2=F.log_softmax(y, dim=1)  

In [62]:
y2

tensor([[-0.8151, -0.5845],
        [-0.7735, -0.6187]], grad_fn=<LogSoftmaxBackward>)

In [113]:
y_train

tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [114]:
x_train

tensor([[ 5,  0],
        [ 6,  0],
        [ 7,  0],
        [ 8,  0],
        [ 9,  0],
        [10,  0],
        [11,  0],
        [12,  0],
        [13, 14],
        [15,  0]])

In [115]:
x=x_train[0:2]

In [116]:
x.shape


torch.Size([2, 2])

In [120]:
y_pred=model(x)

In [121]:
y_pred

tensor([[[0.1373],
         [0.1557]],

        [[0.5417],
         [0.1557]]], grad_fn=<SigmoidBackward>)

In [117]:
y=y_train[0:2]

In [118]:
y

tensor([1, 1])

In [119]:
y.shape

torch.Size([2])

In [189]:
# training process, calculate Gradient Descent                                  
for epoch in range(epochs):                                                     
    i = 0;                                                                      
    print('epoch: ', epoch,' begin .. ')                                        
    # batch feeding the data                                                    
    for i in range(0, x_train.size()[0], batch_size):                           
      # get a batch of data                                                     
      x = x_train[i:i+batch_size]                                               
      y = y_train[i:i+batch_size]     
    
      # make the same size as input
      y = y.view(-1,1)
    
                                                                                
      # Forward pass: Compute predicted y by passing x to the model             
      y_pred = model(x)                                                         
                                                                                
      # Compute and print loss                                                  
      loss = criterion(y_pred, y)                                               
      print('i: ', i,' loss: ', loss.item())                                    
                                                                                
      # Zero gradients, perform a backward pass, and update the weights.        
      optimizer.zero_grad()                                                     
                                                                                
      # perform a backward pass (backpropagation)                               
      loss.backward()                                                           
                                                                                
      # Update the parameters                                                   
      optimizer.step()   

epoch:  0  begin .. 
i:  0  loss:  1.1936943531036377
i:  2  loss:  1.0215333700180054
i:  4  loss:  0.6172489523887634
i:  6  loss:  0.5174863338470459
i:  8  loss:  0.6156891584396362
epoch:  1  begin .. 
i:  0  loss:  1.1685422658920288
i:  2  loss:  0.9991780519485474
i:  4  loss:  0.6139192581176758
i:  6  loss:  0.5307216048240662
i:  8  loss:  0.6239539980888367
epoch:  2  begin .. 
i:  0  loss:  1.144911527633667
i:  2  loss:  0.9782267808914185
i:  4  loss:  0.6110203266143799
i:  6  loss:  0.5434756278991699
i:  8  loss:  0.6318908929824829
epoch:  3  begin .. 
i:  0  loss:  1.1226980686187744
i:  2  loss:  0.9585795402526855
i:  4  loss:  0.6085044145584106
i:  6  loss:  0.5557519197463989
i:  8  loss:  0.6395045518875122
epoch:  4  begin .. 
i:  0  loss:  1.1018050909042358
i:  2  loss:  0.9401436448097229
i:  4  loss:  0.6063287258148193
i:  6  loss:  0.5675560235977173
i:  8  loss:  0.6468002796173096


In [53]:
embedding = nn.Embedding(10, 3)

In [54]:
input = torch.LongTensor([[1,2,4,5]])

In [55]:
t= embedding(input)

In [56]:
t

tensor([[[-2.2279, -1.2578,  0.4777],
         [-2.1180,  0.9439,  0.1689],
         [ 0.2532, -0.6277,  1.7340],
         [-0.4104, -1.5760,  0.9501]]], grad_fn=<EmbeddingBackward>)

In [57]:
t.shape

torch.Size([1, 4, 3])

In [58]:
embedding.num_embeddings

10

In [192]:
embedding.vocab

AttributeError: 'Embedding' object has no attribute 'vocab'